In [1]:
%cd E:\kaggle\iceberg

E:\kaggle\iceberg


In [2]:
# https://www.kaggle.com/dongxu027/explore-stacking-lb-0-1463/notebook
import numpy as np
import pandas as pd

In [3]:
def min_max_best_base():
    file_names = [
        'submission-vgg-mobilenet-0.1620.csv', 
        'submission-vgg-mobilenet-0.1666.csv', 
#         'submission-vgg-mobilenet-0.1696,.csv', 
        'submission-vgg-mobilenet-filtered-0.1515.csv',
#         'submission-vgg-0.1711.csv',
#         'submission-cnn-custom-0.1760.csv',
        'submission-vgg-mobilenet-plain-0.1519.csv',
        'submission-vgg-mobilenet-plain-0.1688.csv'
    ]
    data_len = len(file_names) + 1
    best_base_name = 'submission-vgg-mobilenet-filtered-0.1515.csv'
    outs = [pd.read_csv(f, index_col=0) for f in file_names]
    concat_sub = pd.concat(outs, axis=1)
    cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
    concat_sub.columns = cols
    concat_sub.reset_index(inplace=True)

    concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:data_len].max(axis=1)
    concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:data_len].min(axis=1)
    concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:data_len].mean(axis=1)
    concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:data_len].median(axis=1)

    cutoff_lo = 0.8
    cutoff_hi = 0.2
    print("data len: ", data_len)

    # MinMax + BestBase Stacking
    sub_base = pd.read_csv(best_base_name)
    concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
    concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:data_len] > cutoff_lo, axis=1), 
                                        concat_sub['is_iceberg_max'], 
                                        np.where(np.all(concat_sub.iloc[:,1:data_len] < cutoff_hi, axis=1),
                                                 concat_sub['is_iceberg_min'], 
                                                 concat_sub['is_iceberg_base']))
    concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_bestbase.csv', 
                                            index=False, float_format='%.6f')

    return concat_sub, cols

In [4]:
concat_sub, cols = min_max_best_base()
concat_sub[cols].corr()

data len:  6


,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
is_iceberg_0,1.000000,0.938385,0.974637,0.966892,0.936269
is_iceberg_1,0.938385,1.000000,0.915537,0.961806,0.925594
is_iceberg_2,0.974637,0.915537,1.000000,0.955937,0.919198
is_iceberg_3,0.966892,0.961806,0.955937,1.000000,0.946207
is_iceberg_4,0.936269,0.925594,0.919198,0.946207,1.000000


In [5]:
concat_sub[cols + ["is_iceberg"]].astype(np.str).head(30)

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg
0,0.0327404625714,1.72260097315e-05,0.0283255577087,0.014575409703,0.000550802855287,1.72260097315e-05
1,0.305447280407,0.507933557034,0.4804225564,0.396616488695,0.0473949760199,0.396616488695
2,0.00131119391881,3.58901779691e-05,0.00279717938974,0.000149459665408,0.000115032911708,3.58901779691e-05
3,0.999512970448,0.999996185303,0.997167885303,0.99924248457,0.999974787235,0.999996185303
4,0.0199474655092,0.00696487585083,0.0388489440084,0.0405738055706,0.000364334438927,0.000364334438927
5,0.433148920536,0.162379488349,0.399754285812,0.204521968961,0.488253206015,0.204521968961
6,0.0435323938727,0.00390773639083,0.027299772948,0.0129868201911,0.0056570074521,0.00390773639083
7,0.999732494354,0.999995470047,0.998715519905,0.999906539917,0.999898016453,0.999995470047
8,4.66716955998e-05,8.25171113661e-08,3.52957476935e-06,7.99106292249e-09,4.37470134784e-07,7.99106292249e-09
9,0.000343666353729,7.00255853303e-09,2.6394813176e-05,4.73699977022e-08,4.80557389437e-08,7.00255853303e-09
